In [5]:
from kaggle.api.kaggle_api_extended import KaggleApi
import zipfile
import os

api = KaggleApi()
api.authenticate()

dataset = 'arevel/chess-games'  # Example: 'zillow/zecon'

api.dataset_download_files(dataset, path='.', unzip=True)

Dataset URL: https://www.kaggle.com/datasets/arevel/chess-games


In [ ]:
import sqlite3
import pandas as pd

conn = sqlite3.connect('chess_games.db')
cursor = conn.cursor()

df = pd.read_csv('chess_games.csv')

df.to_sql('Games', conn, if_exists='replace', index=False)

cursor.execute('SELECT * FROM Games LIMIT 5')
print(cursor.fetchall())

[(' Classical ', 'eisaaaa', 'HAMID449', '1-0', '2016.06.30', '22:00:01', 1901, 1896, 11.0, -11.0, 'D10', 'Slav Defense', '300+5', 'Time forfeit', '1. d4 d5 2. c4 c6 3. e3 a6 4. Nf3 e5 5. cxd5 e4 6. Ne5 cxd5 7. Qa4+ Bd7 8. Nxd7 Nxd7 9. Nc3 Nf6 10. Qb3 Be7 11. Nxd5 Qa5+ 12. Nc3 O-O 13. Be2 b5 14. O-O Rad8 15. Bd2 Qc7 16. Rac1 Qd6 17. Qc2 Qe6 18. Nb1 Bd6 19. a3 Nb6 20. Qc6 Nfd5 21. Ba5 Rc8 22. Qb7 Qh6 23. h3 Nc4 24. Bxc4 bxc4 25. Qxd5 Rfd8 26. Qxe4 Rd7 27. Bc3 Re7 28. Qf3 Re6 29. Nd2 Rf6 30. Qg4 Re8 31. Ne4 Rg6 32. Qd7 Rf8 33. Nxd6 Rxd6 34. Qc7 Rg6 35. Qh2 Re8 36. d5 f6 37. d6 Rd8 38. Rfd1 1-0'), (' Blitz ', 'go4jas', 'Sergei1973', '0-1', '2016.06.30', '22:00:01', 1641, 1627, -11.0, 12.0, 'C20', "King's Pawn Opening: 2.b3", '300+0', 'Normal', '1. e4 e5 2. b3 Nf6 3. Bb2 Nc6 4. Nf3 d6 5. d3 g6 6. Nbd2 Bg7 7. g3 Be6 8. Bg2 Qd7 9. O-O O-O 10. c3 b5 11. d4 exd4 12. cxd4 Bg4 13. Rc1 Rfe8 14. Qc2 Nb4 15. Qxc7 Qxc7 16. Rxc7 Nxa2 17. Ra1 Nb4 18. Raxa7 Rxa7 19. Rxa7 Nxe4 20. Nxe4 Rxe4 21. Ng5 Re1+ 

In [7]:
print(df.head())

                Event            White       Black Result     UTCDate  \
0          Classical           eisaaaa    HAMID449    1-0  2016.06.30   
1              Blitz            go4jas  Sergei1973    0-1  2016.06.30   
2   Blitz tournament   Evangelistaizac      kafune    1-0  2016.06.30   
3     Correspondence            Jvayne    Wsjvayne    1-0  2016.06.30   
4   Blitz tournament            kyoday   BrettDale    0-1  2016.06.30   

    UTCTime  WhiteElo  BlackElo  WhiteRatingDiff  BlackRatingDiff  ECO  \
0  22:00:01      1901      1896             11.0            -11.0  D10   
1  22:00:01      1641      1627            -11.0             12.0  C20   
2  22:00:02      1647      1688             13.0            -13.0  B01   
3  22:00:02      1706      1317             27.0            -25.0  A00   
4  22:00:02      1945      1900            -14.0             13.0  B90   

                                         Opening TimeControl   Termination  \
0                                   Sl

In [9]:
query = '''
SELECT Opening, 
       COUNT(*)
FROM Games
WHERE WhiteElo BETWEEN 1200 AND 1600 OR BlackElo BETWEEN 1200 AND 1600
GROUP BY Opening
ORDER BY COUNT(*) DESC
LIMIT 10;
'''

cursor.execute(query)
popular_openings = cursor.fetchall()

for opening in popular_openings:
    print(opening)

("Van't Kruijs Opening", 69116)
('Scandinavian Defense: Mieses-Kotroc Variation', 56369)
('Scandinavian Defense', 43780)
('Modern Defense', 39550)
('Sicilian Defense', 39397)
('Horwitz Defense', 38183)
('Philidor Defense #3', 36863)
('French Defense: Knight Variation', 36750)
("Queen's Pawn Game #2", 33739)
('Sicilian Defense: Bowdler Attack', 32897)


In [ ]:

query = '''
SELECT 
    Event,
    CASE 
        WHEN WhiteElo < 1000 AND BlackElo < 1000 THEN 'Beginner'
        WHEN (WhiteElo BETWEEN 1000 AND 1399) OR (BlackElo BETWEEN 1000 AND 1399) THEN 'Intermediate'
        WHEN (WhiteElo BETWEEN 1400 AND 1799) OR (BlackElo BETWEEN 1400 AND 1799) THEN 'Advanced'
        ELSE 'Expert'
    END AS EloCategory,
    COUNT(CASE WHEN Result = '1/2-1/2' THEN 1 END) * 100.0 / COUNT(*) AS DrawRate
FROM Games
GROUP BY Event, EloCategory
ORDER BY DrawRate DESC
LIMIT 10;
'''

"""
SQL Logical Processing Order

FROM:
WHERE: Filter out.
GROUP BY: Group by.
SELECT: Aggregate functions, data you want to see.
ORDER BY: Sorted.

Syntax: write SELECT first.
"""

cursor.execute(query)

results = cursor.fetchall()

column_names = [description[0] for description in cursor.description]
print(" | ".join(column_names))
print("-" * 50)

for row in results:
    print(" | ".join([str(value) for value in row]))

Event | EloCategory | DrawRate
--------------------------------------------------
Blitz  | Intermediate | 33.333333333333336
Bullet  | Advanced | 20.0
 Correspondence  | Expert | 8.887468030690536
 Correspondence  | Intermediate | 6.573896353166987
 Classical tournament  | Expert | 6.049432739059967
 Blitz tournament  | Expert | 5.593663448347268
 Classical  | Expert | 5.497502063386053
 Blitz  | Expert | 5.309363445643587
 Correspondence  | Advanced | 4.692194591830211
 Bullet tournament  | Expert | 4.331190073322053


In [20]:
conn.close()